In [6]:
import xlwings as xw
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

# Connect to the open workbook "TickerGEX.xlsx"
wb = xw.Book('TickerGEX.xlsx')  # Adjust to the correct file path

# Access the sheet where data is stored, assuming a similar structure to the original SPX data
ws_ticker = wb.sheets['Ticker Data']  # Replace 'Ticker Data' with the actual sheet name if different

# Extract the stock ticker and date-related info from cell A1
info_A1 = ws_ticker.range('A1').value
ticker = info_A1.split(' for ')[1].split(' on ')[0]  # Derive the ticker (e.g., "NVDA")
date_part = info_A1.split(' on ')[1].split(' ')[0]  # Extract the date

# Extract the stock last price from A4
stock_last_price = ws_ticker.range('A4').value

# Extract the expiration date from cell P10
expiration_date_str = ws_ticker.range('P10').value  # Modify this range if the expiration date is stored elsewhere
expiration_date = datetime.strptime(expiration_date_str, '%Y-%m-%d')  # Assuming 'YYYY-MM-DD' format
expiration_week_title = expiration_date.strftime('for the week of %B %d, %Y')  # Format "for the week of [date]"

# Define the start row
start_row = 10

# Automatically detect the last row with useful data (assuming column C reliably contains data)
row = start_row
while True:
    value = ws_ticker.range(f'C{row}').value
    if not value:  # Stop when an empty cell is found
        end_row = row - 1  # Set the end row to the last non-empty row
        break
    row += 1

print(f"Start row: {start_row}, End row: {end_row}")

# Initialize lists to store strikes and gamma exposures for Calls and Puts
strikes = []
call_gamma_exposure_list = []
put_gamma_exposure_list = []
call_volumes = []
put_volumes = []

# Loop through each row and extract relevant data for Calls and Puts
for row in range(start_row, end_row + 1):
    strike_price = ws_ticker.range(f'Q{row}').value
    call_gamma = ws_ticker.range(f'I{row}').value
    call_open_int = ws_ticker.range(f'K{row}').value
    put_gamma = ws_ticker.range(f'AB{row}').value
    put_open_int = ws_ticker.range(f'AD{row}').value
    call_volume = ws_ticker.range(f'G{row}').value  # Volume for Calls
    put_volume = ws_ticker.range(f'Z{row}').value  # Volume for Puts

    if call_gamma and call_open_int:
        call_gamma_exposure = call_gamma * call_open_int * (stock_last_price)**2 * 0.01
    else:
        call_gamma_exposure = 0

    if put_gamma and put_open_int:
        put_gamma_exposure = put_gamma * put_open_int * (stock_last_price)**2 * 0.01 * -1
    else:
        put_gamma_exposure = 0

    strikes.append(strike_price)
    call_gamma_exposure_list.append(call_gamma_exposure)
    put_gamma_exposure_list.append(put_gamma_exposure)
    call_volumes.append(call_volume)
    put_volumes.append(put_volume)

# Define the ±350 range from the stock last price
min_strike = stock_last_price - 50
max_strike = stock_last_price + 50

# Create a DataFrame
df = pd.DataFrame({
    'Strike': strikes,
    'Call Gamma Exposure': call_gamma_exposure_list,
    'Put Gamma Exposure': put_gamma_exposure_list,
    'Call Volume': call_volumes,
    'Put Volume': put_volumes
})

# Filter the DataFrame to only show strikes within the ±350 range of stock last price
df_filtered = df[(df['Strike'] >= min_strike) & (df['Strike'] <= max_strike)].copy()

# Use .loc[] to avoid SettingWithCopyWarning
df_filtered.loc[:, 'Total Gamma Exposure'] = df_filtered['Call Gamma Exposure'] + df_filtered['Put Gamma Exposure']

# ---- Find Gamma Flip Point ----
total_gamma_filtered = df_filtered['Total Gamma Exposure'].values
levels_filtered = df_filtered['Strike'].values
zero_cross_idx = np.where(np.diff(np.sign(total_gamma_filtered)))[0]

# Check if there's any zero-crossing index
if zero_cross_idx.size > 0:
    # Store potential flip points
    flip_points = []
    for idx in zero_cross_idx:
        neg_gamma = total_gamma_filtered[idx]
        pos_gamma = total_gamma_filtered[idx + 1]
        neg_strike = levels_filtered[idx]
        pos_strike = levels_filtered[idx + 1]

        # Linear interpolation to find the exact Gamma Flip point
        interpolated_flip = pos_strike - ((pos_strike - neg_strike) * pos_gamma / (pos_gamma - neg_gamma))
        flip_points.append(interpolated_flip)

    # If multiple flip points, find the one closest to the stock last price
    flip_points = np.array(flip_points)
    closest_flip_idx = np.argmin(np.abs(flip_points - stock_last_price))
    zero_gamma = flip_points[closest_flip_idx]
else:
    zero_gamma = None
    print("Warning: No Gamma Flip Point found. Skipping the Gamma Flip line in the chart.")

# ---- Chart Creation Section ----

# Initialize the figure
fig = go.Figure()

# Create the bar chart (Gamma Exposure: Call and Put)
bar_chart_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Gamma Exposure'], name='Call Gamma', marker_color='blue')
bar_chart_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Gamma Exposure'], name='Put Gamma', marker_color='red')

# Create the bar chart (Total Gamma Exposure)
bar_chart_total = go.Bar(x=df_filtered['Strike'], y=df_filtered['Total Gamma Exposure'], name='Total Gamma', marker_color='green')

# Create the volume distribution chart (Volume by Strike Price)
volume_call = go.Bar(x=df_filtered['Strike'], y=df_filtered['Call Volume'], name='Call Volume', marker_color='blue')
volume_put = go.Bar(x=df_filtered['Strike'], y=df_filtered['Put Volume'], name='Put Volume', marker_color='red')

# Add a vertical line for Stock Last Price
y_min = min(df_filtered['Call Gamma Exposure'].min(), df_filtered['Put Gamma Exposure'].min())
y_max = max(df_filtered['Call Gamma Exposure'].max(), df_filtered['Put Gamma Exposure'].max())
spot_price_line = go.Scatter(
    x=[stock_last_price, stock_last_price],  # Constant x (Stock Last Price)
    y=[y_min, y_max],  # Updated y-range
    mode='lines',
    name=f'{ticker} Last Price: {stock_last_price}',
    line=dict(color='black', width=3, dash='solid')  # Style the line
)

# ---- Add Gamma Flip line to all charts ----
if zero_gamma:
    gamma_flip_line = go.Scatter(
        x=[zero_gamma, zero_gamma],  # Constant x (Gamma Flip Strike)
        y=[y_min, y_max],  # Updated y-range
        mode='lines',
        name=f'Gamma Flip: {zero_gamma}',
        line=dict(color='purple', width=3, dash='dash')  # Style the line
    )

# Add traces to the figure
fig.add_traces([bar_chart_call, bar_chart_put, volume_call, volume_put, bar_chart_total, spot_price_line])

# Add Gamma Flip line to all charts
if zero_gamma:
    fig.add_trace(gamma_flip_line)

# Define the dropdown menu options
dropdown_buttons = [
    dict(label='Gamma Exposure (Call/Put)',
         method='update',
         args=[{'visible': [True, True, False, False, False, True]},
               {'title': f'Gamma Exposure by Strike Price (Call/Put) - {ticker} - {expiration_week_title}'}]),
    dict(label='Total Gamma Exposure',
         method='update',
         args=[{'visible': [False, False, False, False, True, True]},
               {'title': f'Total Gamma Exposure by Strike Price - {ticker} - {expiration_week_title}'}]),
    dict(label='Volume Distribution',
         method='update',
         args=[{'visible': [False, False, True, True, False, True]}, 
               {'title': f'Volume Distribution by Strike Price - {ticker} - {expiration_week_title}'}]),
]

# Initialize all traces to be invisible except the default (Call/Put Gamma Exposure bar chart)
fig.update_traces(visible=False)
fig.data[0].visible = True  # Call Gamma (Bar)
fig.data[1].visible = True  # Put Gamma (Bar)
fig.data[5].visible = True  # Stock Last Price Line
if zero_gamma:
    fig.data[-1].visible = True  # Gamma Flip line

# Add dropdown menu
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=dropdown_buttons,
        x=1.15,  # Positioning the dropdown
        y=1.15,
        xanchor='right'
    )]
)

# Update layout
fig.update_layout(
    title=f'Gamma Exposure by Strike Price (Call/Put) - {ticker} - {expiration_week_title}',
    xaxis_title='Strike Price',
    yaxis_title='Gamma Exposure',
    barmode='group',
    xaxis_tickangle=-45
)

# Show the figure with dropdown
fig.show()


Start row: 10, End row: 86
